In [1]:
import sys
sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from interpolant.cell_to_points import *
from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5

In [3]:
np.set_printoptions(linewidth=500)

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))


gm.draw_grid()

In [4]:
cell15 = cell.Cell2D(size=(1,1), ll_vertex=(0,8))
cell25 = cell.Cell2D(size=(1,1), ll_vertex=(0,9))

cell14 = cell.Cell2D(size=(1,1), ll_vertex=(0,6))
cell24 = cell.Cell2D(size=(1,1), ll_vertex=(0,7))

cell13 = cell.Cell2D(size=(1,1), ll_vertex=(0,4))
cell23 = cell.Cell2D(size=(1,1), ll_vertex=(0,5))

cell12 = cell.Cell2D(size=(1,1), ll_vertex=(0,2))
cell22 = cell.Cell2D(size=(1,1), ll_vertex=(0,3))

cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))
cell32 = cell.Cell2D(size=(2,2), ll_vertex=(1,2))
cell33 = cell.Cell2D(size=(2,2), ll_vertex=(1,4))
cell34 = cell.Cell2D(size=(2,2), ll_vertex=(1,6))
cell35 = cell.Cell2D(size=(2,2), ll_vertex=(1,8))

In [5]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2, cell12, cell22, cell13, cell23, cell14, cell24, cell15, cell25]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3, cell32, cell33, cell34, cell35]], layer_number=1, index=[1], div_index=[1])

#layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
#layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [6]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [7]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()

In [ ]:
#6 cells
ma._vertex_ddof_index[((0, 3), (0, 4))] = (ma._vertex_ddof_index[((0, 3), (0, 4))][0], ma._vertex_ddof_index[((0, 0), (0, 0))][1])
ma._vertex_ddof_index[((0, 3), (1, 4))] = (ma._vertex_ddof_index[((0, 3), (1, 4))][0], ma._vertex_ddof_index[((0, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (1, 4))] = (ma._vertex_ddof_index[((1, 2), (1, 4))][0], ma._vertex_ddof_index[((1, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (3, 4))] = (ma._vertex_ddof_index[((1, 2), (3, 4))][0], ma._vertex_ddof_index[((1, 0), (3, 0))][1])

In [ ]:
#3 cells
ma._vertex_ddof_index[((0, 1), (0, 2))] = (ma._vertex_ddof_index[((0, 1), (0, 2))][0], ma._vertex_ddof_index[((0, 0), (0, 0))][1])
ma._vertex_ddof_index[((0, 1), (1, 2))] = (ma._vertex_ddof_index[((0, 1), (1, 2))][0], ma._vertex_ddof_index[((0, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 0), (1, 2))] = (ma._vertex_ddof_index[((1, 0), (1, 2))][0], ma._vertex_ddof_index[((1, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 0), (3, 2))] = (ma._vertex_ddof_index[((1, 0), (3, 2))][0], ma._vertex_ddof_index[((1, 0), (3, 0))][1])

In [ ]:
#6 cells
edge_mapping = {((0, 4), (1, 4)): ((0, 0), (1, 0)), ((1, 4), (3, 4)): ((1, 0), (3, 0))}
#3cells
#edge_mapping = {((0, 2), (1, 2)): ((0, 0), (1, 0)), ((1, 2), (3, 2)): ((1, 0), (3, 0))}
for (cell, edge), dd in ma._edge_ddof_index.items():
    if edge_mapping.get(edge) is not None:
        edge_slave = edge
        edge_prim = edge_mapping[edge]
        prim_cell = [i for i in list(ma._edge_ddof_index.keys()) if i[1] == edge_prim][0][0]
        ma._edge_ddof_index[(cell, edge_slave)] = [(i[0],j[1])
            for i,j in
            zip(ma._edge_ddof_index[(cell, edge_slave)], ma._edge_ddof_index[(prim_cell, edge_prim)])
        ]

In [ ]:
#6 cells
ma._weak_edge_connections[ ((0, 2),
  ((1, 2), (1, 3)))] = {((1, 2), ((1, 2), (1, 4))): [(0, 54),
   (1, 74),
   (2, 75),
   (3, 61)]}

ma._weak_edge_connections[((0, 3),
  ((1, 3), (1, 4)))] = {((1, 2), ((1, 2), (1, 4))): [(0, 54),
   (1, 74),
   (2, 75),
   (3, 61)]}

ma._weak_edge_connections[((1, 2),
  ((1, 2), (1, 4)))] = {((0, 2), ((1, 2), (1, 3))): [(12, 19),
   (13, 34),
   (14, 35),
   (15, 31)], ((0, 3), ((1, 3), (1, 4))): [(12, 31),
   (13, 46),
   (14, 47),
   (15, 9)]}

In [ ]:
#3cells
ma._weak_edge_connections[((0, 0), ((1, 0), (1, 1)))] = {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 37)]}
ma._weak_edge_connections[((0, 1), ((1, 1), (1, 2)))] = {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 37)]}

ma._weak_edge_connections[((1, 0),((1, 0), (1, 2)))] = {((0, 0), ((1, 0), (1, 1))): [(12, 9),
   (13, 10),
   (14, 11),
   (15, 3)], ((0, 1), ((1, 1), (1, 2))): [(12, 3),
   (13, 22),
   (14, 23),
   (15, 9)]}

In [8]:
grid_domain = GridDomain(domain_size=(3.,10.), integer_size=(3,10))
grid_domain.make_pointwise_index(ma)

In [ ]:
grid_domain.pointwise_index

In [9]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)

In [10]:
deep = MA.assemble()

[(0, 4), (1, 42), (2, 43), (3, 44), (4, 45), (5, 40), (6, 0), (7, 50), (8, 51), (9, 52), (10, 53), (11, 36), (12, 1), (13, 54), (14, 55), (15, 56), (16, 57), (17, 37), (18, 2), (19, 58), (20, 59), (21, 60), (22, 61), (23, 38), (24, 3), (25, 62), (26, 63), (27, 64), (28, 65), (29, 39), (30, 5), (31, 46), (32, 47), (33, 48), (34, 49), (35, 41)] [(0, 10), (1, 6), (2, 7), (3, 8), (4, 9), (5, 4), (6, 11), (7, 20), (8, 21), (9, 22), (10, 23), (11, 0), (12, 12), (13, 24), (14, 25), (15, 26), (16, 27), (17, 1), (18, 13), (19, 28), (20, 29), (21, 30), (22, 31), (23, 2), (24, 14), (25, 32), (26, 33), (27, 34), (28, 35), (29, 3), (30, 15), (31, 16), (32, 17), (33, 18), (34, 19), (35, 5)]
{0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65} {306, 307, 308, 309, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 326, 327, 328, 329, 330, 331, 3

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [11]:
def partial_diff_test3(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True, scale_factor=1):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der * scale_factor))
    print('Y derivative: {}'.format(y_der * scale_factor))
    print('XX derivative: {}'.format(xx_der * scale_factor))
    print('YY derivative: {}'.format(yy_der * scale_factor))
    print('XY_derivative: {}'.format(xy_der * scale_factor), end='\n\n')

In [12]:
def partial_diff_test2(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')
    return (xx_der + yy_der)/2

In [ ]:
mm = np.zeros(MA.half_glob.shape[0])
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    mm[i] = partial_diff_test2(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
MM = csr_matrix(np.diag(-1/mm))

In [ ]:
for i in sorted([142, 158, 159, 154, 19, 34, 35, 31, 46, 47, 43]):
    print(i, end=' ')
    partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
2 * (0.007416255387549075 - 0.023370207540036145 + 0.0892035115259431)

In [23]:
for num,i in enumerate([310, 352, 353, 354, 355, 346]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.half_glob,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       scale_factor=1/w_ld[num] * MA.I_s2b[5][num]*w_rd[5]
                       #scale_factor=1
)

310 X derivative: 0.00833302801058048
Y derivative: 1.939362147940258e-07
XX derivative: -0.0005555509630025775
YY derivative: -0.0005568500612148363
XY_derivative: -3.568870349245592e-12

352 X derivative: -0.012976497701529352
Y derivative: 7.082307442331271e-07
XX derivative: 0.0008650178030095891
YY derivative: 0.0008643426763478364
XY_derivative: 5.2445245865646075e-12

353 X derivative: 0.04214320801563697
Y derivative: 1.256658749090653e-06
XX derivative: -0.0028096186444540212
YY derivative: -0.0028101692084408513
XY_derivative: 2.526622442273526e-11

354 X derivative: 0.04214343209913484
Y derivative: -1.4443588412515666e-06
XX derivative: -0.0028096758350916228
YY derivative: -0.0028097017535536417
XY_derivative: -5.654009277508047e-12

355 X derivative: -0.012976096650456069
Y derivative: -6.865353746795471e-07
XX derivative: 0.0008651101561976906
YY derivative: 0.0008644576953951478
XY_derivative: -2.461022234928993e-13

346 X derivative: 0.008333028010580482
Y derivative: 

In [22]:
for num,i in enumerate([131]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.half_glob,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       scale_factor=1/2
)

131 X derivative: -0.03333420639203072
Y derivative: 3.440175782867053e-07
XX derivative: -0.0011110552091573556
YY derivative: -0.0011101928953796485
XY_derivative: -1.3645056015260279e-11



In [ ]:
MA.I_s2b

In [ ]:
for num,i in enumerate([142, 158, 159, 154]):
    print(i, end=' ')
    partial_diff_test3(matrix=MM*MA.half_glob,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       scale_factor=1/w_ld[num] * MA.I_b2s.T[3][num] * w_rd[3]
)

In [ ]:
w_ld

In [15]:
for num,i in enumerate([101, 136, 137, 138, 139, 131, 166, 167, 168, 169, 161]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.assembled,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       #scale_factor=1/w_ld[num] * MA.I_s2b[5][num]/2
                       
                       #scale_factor=1
    )
    #print('norma_val: ', 1/w_ld[num] * MA.I_s2b[5][num]/2)

101 X derivative: -2.094349708809651e-06
Y derivative: 1.1197951683212645e-06
XX derivative: -0.0033332590611676555
YY derivative: 0.00882773239035427
XY_derivative: -4.957917565874049e-07

136 X derivative: -1.3159327772577743e-06
Y derivative: 0.047632575491180815
XX derivative: -0.009462264974047799
YY derivative: -0.017970763633895003
XY_derivative: -1.4463834451833193e-06

137 X derivative: 2.2981040737526243e-06
Y derivative: -0.0426781678950671
XX derivative: -0.013870246461415608
YY derivative: -0.007252780659856188
XY_derivative: 2.5436727417759483e-06

138 X derivative: -4.428063639849364e-07
Y derivative: 0.029948731836517244
XX derivative: -0.01387178252317869
YY derivative: -0.032299520383136486
XY_derivative: -1.9422177495356362e-06

139 X derivative: 1.8578000718767708e-07
Y derivative: -0.02188738784080675
XX derivative: -0.009461649473615667
YY derivative: 0.0026976714136758033
XY_derivative: 9.069946252102177e-07

131 X derivative: 5.846445268165002e-07
Y derivative: 

In [ ]:
MA.I_b2s.T[3]

In [ ]:
w_ld

In [ ]:
print(deep[0][1].T[5].nonzero())
deep[0][1].T[5].toarray().flatten().take(deep[0][1].T[5].nonzero()[1])[1:]

In [ ]:
print(deep[0][1][5].nonzero())
deep[0][1][5].toarray().flatten().take(deep[0][1][5].nonzero()[1])

In [ ]:
MA.I_s2b[5]

In [ ]:
np.dot(np.dot(w_r, MA.I_s2b), np.linalg.inv(w_l))

In [ ]:
MA.I_b2s.T

In [16]:
c2p = Cell2PointsConverter(orders=[[5],[5,5]])

In [17]:
w_rd

NameError: name 'w_rd' is not defined

In [18]:
w_ld, w_rd = c2p.get_weights()

In [19]:
w_l, w_r = c2p.get_diag_weights()

In [ ]:
grid_domain.normal_index[82]

In [ ]:
for i in [81, 82, 83, 84, 85, 70]:
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
0.07337582303605292/ -0.018924022932711504

In [ ]:
f = h5py.File('../datasources/1_10_globatto_matrices.h5')

In [ ]:
sc = f['mass_matrices/base_M_order_5'][()]

In [ ]:
mm_new = np.diag(sc.sum(axis=1))

In [ ]:
mm_new

In [ ]:
init_op = MA.half_glob * linalg.inv(deep[0][1])

In [ ]:
init_op

In [ ]:
MA.I_s2b[5]